# How do I _copy_ an app from the _public apps_?
### Overview
Apps can be copied from one of two sources:
1. other projects you are a member of
2. public reference. 

Here we focus on copying an app from _public reference_. 

### Prerequisites
 1. You need to be a member (or owner) of _at least **one**_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="set_AUTH_TOKEN.ipynb">**set_AUTH_TOKEN.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="apps_listAll.ipynb" target="_blank">list</a> apps within your project (we will just use that call directly and pick one here).
 
### Imports and Definitions
A single call is sufficient to copy an app. Here there is **no advantage** in using the object to POST the call, but it is rather useful searching the results of the GET calls.

In [ ]:
from defs.apimethods import *

## Use API() _object_
We will first list all our projects, then list the apps within the Public Reference project<sup>1</sup> , and finally copy an app between the Public Reference to the target project. Here we also explicitly set _'limit':100_ inside the _query_. This helps speed up the auto-pagination feature within the object constructor.

The critical information for this POST is the **app_id**. Note, you are **not** allowed to copy the same app **and** assign the same name more than once. If you change the name, it is ok.

To make these results very obvious, use an empty project as your TARGET\_PROJECT (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the _name_ in the _data_ dictionary to something like 'Dept of Awesome'<sup>2</sup>


<sup>1</sup> Unlike files, apps are accesssible **both** *within* a project (here the Public Reference project) **and** by a *visibility* property (which may be set to 'public')

<sup>2</sup> Note that setting the **name** of an app, actually changes the **id**. We are working on fixing this inconsistency.

#### Why am I getting an Error Code 6000?
You found a known bug, we are _very sorry_ for this. It is already being worked on and should be ok soon. Try copying the app named **'Torrent Mapping Alignment Program Map4'**, this will work. Known affected workflows:
* Exome Coverage QC 1.0
* RNA-Seq De Novo Assembly and Analysis - Trinity 2.0.6
* RNA-Seq De Novo Assembly - Trinity 2.0.6
* Whole Exome Sequencing GATK 2.3.9.-lite

In [ ]:
# [USER INPUT] Set project (p_) index and app name:
p_index = 1                                       # target project
a_name = 'Torrent Mapping Alignment Program Map4'

# LIST all projects
existing_projects = API('projects')         

# LIST all Public Apps using VISIBILITY and searching by NAME
my_apps_source = API(path='apps', query={'visibility': 'public', 'limit': 100})
my_apps_target = API(path='apps', query={'project':existing_projects.id[p_index]})
a_index = my_apps_source.name.index(a_name)

# Check if app already exists in the second project
if my_apps_source.id[a_index] in my_apps_target.id:
    print('App already exists in second project, please try another app')
else:
    print('App (%s) does not exist in Project (%s); copying now' % \
          (my_apps_source.name[a_index], existing_projects.id[p_index]))
    
    # COPY the selected app from first to second project
    API(path=('apps/' + my_apps_source.id[a_index] + '/actions/copy'), \
        method='POST', \
        data={'project': existing_projects.id[p_index],\
              'name': my_apps_source.name[a_index]})      # 'name': 'Dept of Awesome'})

    # re-list the apps in secondProject to verify the copy worked
    my_apps_target = API(path='apps', \
                         query={'project': existing_projects.id[p_index]})
    
    if my_apps_source.name[a_index] in my_apps_target.name:
        print('Sucessfully copied one app!')
    else:
        print('Something went wrong...')
    
    # Note for Footnote #2
    print(my_apps_target.name)
    print(my_apps_target.id)

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-an-app-secondary-method)